In [1]:
import mediapipe as mp
import cv2

# 얼굴에서 특징점을 찾을 수 있는 기능 불러오기
mp_face_mesh = mp.solutions.face_mesh
# 찾은 특징점 표현하기
mp_drawing = mp.solutions.drawing_utils
# 특징점 그리기 속성
drawing_spec = mp_drawing.DrawingSpec(thickness = 1, color = (0,0,255))
# 얼굴에서 특징점 찾기 속성
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence = 0.5, # 얼굴 검출 확률(자체 판단) 50% 이상인 것들만 출력하기
    min_tracking_confidence = 0.5  # 특징점 검출 확률(자체 판단) 50% 이상인 것들만 출력하기
)
# 한 번만 실행하면 되는 코드
cat = cv2.imread('images/doong.png')
cat = cv2.resize(cat, (200, 200)) # 123, 132 > 200, 200 으로 크기 변경
mask2gray = cv2.cvtColor(cat , cv2.COLOR_BGR2GRAY)
_, mask_b = cv2.threshold(mask2gray, 200, 255, cv2.THRESH_BINARY)
mask_b_inv = cv2.bitwise_not(mask_b)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 카메라에서 불러온 사진에서 얼굴 특징점 찾기
    results = face_mesh.process(img)

    # 얼굴의 특징점(478개)을 감지했을 때 실행하겠다
    if results.multi_face_landmarks is not None:
        #print(results.multi_face_landmarks)
        #for res in results.multi_face_landmarks:
        #    mp_drawing .draw_landmarks(img, res, mp_face_mesh.FACEMESH_TESSELATION)
        # 478개 점을 사용하기가 편함
        face_landmarks = results.multi_face_landmarks[0]
        
        # 코의 좌표 찾기
        x = int(face_landmarks.landmark[4].x * img.shape[1])
        y = int(face_landmarks.landmark[4].y * img.shape[0])
        # cv2.circle(img, (x,y), 20, (0,0,255), cv2.FILLED) # 원하는 위치에 점을 찍어주는 함수
        # 이미지를 집어넣을 주심 좌표(코의 위치) > x,,y
        # 마스크 이미지의 크기 > 123,132(tiger.shape)

        sub_img = img[y-100 : y+100, x-100 : x+100]
        img_bg = cv2.bitwise_and(sub_img, sub_img, mask = mask_b)
        img_fg = cv2.bitwise_and(cat,cat, mask = mask_b_inv)
        bg_fg = cv2.add(img_bg,img_fg)
        img[y-100 : y+100, x-100 : x+100] = bg_fg

            
    cv2.imshow('video',img)
    if cv2.waitKey(33) == 49:
        break 
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2

psy = cv2.imread('images/psy.png')
tiger = cv2.imread('images/tiger.png')

mask2gray = cv2.cvtColor(tiger , cv2.COLOR_BGR2GRAY)
_, mask_b = cv2.threshold(mask2gray, 200, 255, cv2.THRESH_BINARY)
mask_b_inv = cv2.bitwise_not(mask_b)
sub_psy = psy[ 94:217 , 239:371]
img_bg = cv2.bitwise_and(sub_psy, sub_psy, mask = mask_b)
img_fg = cv2.bitwise_and(tiger,tiger, mask = mask_b_inv)
bg_fg = cv2.add(img_bg,img_fg)
psy[ 94:217 , 239:371] = bg_fg

cv2.imshow('psy',psy)
cv2.waitKey(0)
cv2.destroyAllWindows()